<a href="https://colab.research.google.com/github/GuihJF/python_for_data_analysis/blob/main/Brazilian_public_travel_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Project Introduction:
# Real data on Brazilian public travel expenses by job position.

# Author Details:
name = "Guilherme"
age = 25
country = "Brazil"
current_role = "Service Desk Specialist – Global L1 Support"
aspiration = "Data Analyst / Data Engineer"
skills = ["Python", "Excel", "SQL", "Power BI", "Tech Support"]

# Project Focus:
project_title = "Python for Data: From Scratch to Complete Analysis"
project_goal = (
    "Applying practical Python skills to read, clean, analyze, and interpret data. "
    "This notebook demonstrates my ability to work with real-world datasets using essential data analysis tools and functions."
)